In [2]:
from ipyleaflet import *
import s3fs
import pyarrow as pa
import pyarrow.dataset as ds

import sys
import os

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import Window

import pandas as pd
import polars as pl
import altair as alt
import matplotlib.pyplot as plt

import geopandas as gpd

import datetime

from pyspark.sql.functions import *

In [9]:
m = Map(center=(40.7128, -74.0060), zoom=14)

click_pos=[]
markers=[]
def handle_click(**kwargs):
    if kwargs.get('type') == 'click':
        markers.append(Marker(location=kwargs.get('coordinates')))
        m.add_layer(markers[-1])
        if len(markers)>2:
            m.remove_layer(markers[0])
            markers.pop(0)
            click_pos.pop(0)
        click_pos.append(kwargs.get('coordinates'))
        print(kwargs.get('coordinates'))

m.on_interaction(handle_click)

m

Map(center=[40.7128, -74.006], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zo…

[40.714476284709335, -74.00450706481935]
[40.720136007355364, -73.99446487426759]
[40.717273679029205, -73.99206161499025]
[40.71493168249225, -73.99849891662599]
[40.71675324247426, -74.00407791137697]
[40.72098167171645, -73.99618148803712]
[40.71974569710356, -73.98038864135744]
[40.71506179557237, -74.00150299072267]
[40.72091662099322, -74.00639533996583]
[40.71942043681214, -73.98425102233888]
[40.71408594127328, -73.99446487426759]
[40.71863980562839, -74.00158882141115]
[40.72280306615735, -73.98699760437013]
[40.716102691059234, -73.9948081970215]
[40.718834964282536, -74.00776863098146]
[40.72208752428535, -73.99360656738283]
[40.7152569647158, -73.99686813354494]
[40.716818297266144, -73.98365020751955]
[40.72091662099322, -73.99360656738283]
[40.719290332250566, -74.0075969696045]
[40.71408594127328, -73.99849891662599]
[40.71863980562839, -73.9896583557129]
[40.7213069243792, -73.99970054626466]
[40.71785916529029, -74.00699615478517]
[40.715452133287066, -73.9859676361084